This code attempts to apply XGBoosting to the nutrition dataset to predict the carbon intensity of food products. 
First attempt with no tuning saw a significant decrease in absolute error of 0.2133, for std of 1.61. 
Next step was to apply the whole data set, including Nulls to try and take advantage of SGBoosts characteristic of being able to predict for missing data values. This improved the absolute error by 0.06, however every time the model runs it gets a slightly different error, due to training on a slightly different set. 
So far I have used SGBoost by setting the objective to be reg:squared error, Specify the learning task and the corresponding learning objective to be regression with squared loss. 
Squaredlogerror, uses regression with squared log loss, this sees a mean absolute error of 0.2534 for a sd of y_pred of 1.7. Using this squared log error uses more trees to converge. 
pseudohubererror has an error absolute mean of 0.408, largest so far, probably as it is regression with Pseudo Huber loss, a twice differentiable alternative to absolute loss. Converges to this error around the same as previous iterations, at 20 trees. 


In [2]:
#importing modules for the implementation
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
import xgboost as xgb
from xgboost import XGBRegressor


In [3]:
nutrition = pd.read_csv("./Nutrition_Full_Features.csv")
nutrition = nutrition.dropna()

In [4]:
#dropping NANSinclude a lot of NaNs
#nutrition.drop(nutrition.columns[1:3], axis=1, inplace=True)

In [12]:
#extracting the nutrition data, labelling X, for null data set 
#X = nutrition.iloc[:,16:]
#extracting the greenhouse gas emissions 
y = nutrition.iloc[:,7:8]
y
#y = np.ravel(y)

,GHG(kgco2eq/100g)
1,0.1441
2,0.1441
5,0.1441
9,0.1441
10,0.1441
...,...
1218,0.0455
1219,0.0455
1220,2.1240
1221,0.0455


In [13]:
#extracting the nutrition data, labelling X as this is the input to the SKlearn algorithm
X = nutrition.iloc[:,11:]
X


,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Saturated Fats (g),Calcium (mg),"Iron, Fe (mg)","Potassium, K (mg)",...,"Carotene, alpha (mcg)",Lycopene (mcg),Lutein + Zeaxanthin (mcg),"Fatty acids, total monounsaturated (mg)","Fatty acids, total polyunsaturated (mg)",20:5 n-3 (EPA) (mg),22:5 n-3 (DPA) (mg),22:6 n-3 (DHA) (mg),Caffeine (mg),Theobromine (mg)
1,273,9.22,6.58,41.05,4.30,2.2,1.898,279,6.04,126.0,...,0.0,0.0,63.0,4530.0,1445.0,12.0,0.0,7.0,0.0,0.0
2,309,9.49,7.42,48.39,4.41,2.6,2.275,299,6.59,138.0,...,0.0,0.0,66.0,5292.0,1502.0,13.0,0.0,8.0,0.0,0.0
5,290,11.50,2.17,44.54,22.02,1.6,2.313,14,0.91,76.0,...,0.0,1.0,42.0,5797.0,2117.0,0.0,0.0,0.0,0.0,0.0
9,238,2.15,10.66,43.91,5.00,9.2,0.630,684,4.89,127.0,...,0.0,0.0,25.0,393.0,973.0,3.0,0.0,0.0,0.0,0.0
10,250,1.53,10.20,48.89,6.12,4.1,0.000,20,2.76,165.0,...,0.0,0.0,88.0,290.0,936.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218,241,10.16,1.66,40.60,19.10,2.9,1.507,4,0.78,572.0,...,418.0,0.0,29.0,4099.0,4079.0,0.0,0.0,0.0,0.0,0.0
1219,19,0.27,1.39,3.78,0.71,3.1,0.053,62,0.90,327.0,...,0.0,0.0,4204.0,7.0,126.0,0.0,0.0,0.0,0.0,0.0
1220,96,6.84,5.23,4.32,1.89,1.2,1.486,70,1.15,269.0,...,12.0,313.0,4097.0,2402.0,2112.0,0.0,0.0,0.0,0.0,0.0
1221,208,15.70,1.22,17.17,11.23,2.3,2.452,30,0.49,309.0,...,2157.0,1.0,162.0,3498.0,9319.0,0.0,0.0,1.0,0.0,0.0


In [14]:
#splitting the dataset into a test and train data set, with 20% being used for testing
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X,y,test_size=0.2) #size =0.2 means 80% data is training data, 20% testing


In [15]:
#splitting train data set into train and validation set 
X_train, X_val, y_train, y_val = train_test_split(X_train_val,y_train_val,test_size=0.01)

In [16]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [17]:
from sklearn.metrics import mean_absolute_error

In [26]:
model = XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.24835239447559182
Mean Squared Error: 0.5506567777601412
Root Mean Squared Error: 0.7420625160726967


In [19]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [20]:
params['eval_metric'] = "mae"


In [22]:
num_boost_round = 999

In [24]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

[17:36:36] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:0.75802
[1]	Test-mae:0.61015
[2]	Test-mae:0.50286
[3]	Test-mae:0.44250
[4]	Test-mae:0.38802
[5]	Test-mae:0.35342
[6]	Test-mae:0.32539
[7]	Test-mae:0.30561
[8]	Test-mae:0.29345
[9]	Test-mae:0.28192
[10]	Test-mae:0.27792
[11]	Test-mae:0.27261
[12]	Test-mae:0.27016
[13]	Test-mae:0.26704
[14]	Test-mae:0.26584
[15]	Test-mae:0.26543
[16]	Test-mae:0.26507
[17]	Test-mae:0.26291
[18]	Test-mae:0.26203
[19]	Test-mae:0.26100
[20]	Test-mae:0.25984
[21]	Test-mae:0.25887
[22]	Test-mae:0.25857
[23]	Test-mae:0.25797
[24]	Test-mae:0.25713
[25]	Test-mae:0.25661
[26]	Test-mae:0.25575
[27]	Test-mae:0.25530
[28]	Test-mae:0.25461
[29]	Test-mae:0.25385
[30]	Test-mae:0.25372
[31]	Test-mae:0.25359
[32]	Test-mae:0.25335
[33]	Test-mae:0.25268
[34]	Test-mae:0.25211
[35]	Test-mae:0.25189
[36]	Test-mae:0.25184
[37]	Test-mae:0.25158
[38]	Test-mae:0.25

In [27]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

[17:39:35] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[17:39:35] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[17:39:35] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[17:39:35] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[17:39:35] WARNING: d:\bld\xgboost-split_1615294821523\work\src\objective\regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,0.601116,0.029665,0.640256,0.129425
1,0.447660,0.024377,0.525593,0.105084
2,0.336806,0.017526,0.447917,0.092064
3,0.256870,0.016493,0.389790,0.073854
4,0.199061,0.013470,0.350315,0.065467
...,...,...,...,...
95,0.001085,0.000228,0.270117,0.042205
96,0.001037,0.000216,0.270111,0.042199
97,0.000999,0.000212,0.270113,0.042200
98,0.000962,0.000197,0.270104,0.042195


In [ ]:
# https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f